In [1]:
########################################################################################################################
# This script performs the standard train-test partitioning to stratify the subjects by the target label. 
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# TARGET_DIR_PATH: Path of the directory storing the target datasets (created in P01_Subject_Inclusion.ipynb)
########################################################################################################################
TARGET_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# TEST_SIZE: A float between 0 and 1 representing the proportion of the held-out test partition
########################################################################################################################
TEST_SIZE: float = 0.3

In [3]:
########################################################################################################################
# Define a function to return the target distribution 
########################################################################################################################
def target_dist(df_in: pd.DataFrame, 
                col: str):
    assert col in df_in.columns
    v_count: dict[int, int] = df_in[col].value_counts().sort_index().to_dict()
    v_count_rate: dict[int, float] = df_in[col].value_counts(normalize=True).sort_index().to_dict()
    dist: dict[int, tuple[int, float]] = {k: (v_count[k], f'{v_count_rate[k]*100:.2f}%') for k in v_count.keys()}
    dist |= {'Total': (df_in.shape[0], '100.0%')}
    return f'Distribution of the target: {dist}'

In [ ]:
########################################################################################################################
# Define the column name storing the target label
########################################################################################################################
target_col: str = 'OutcomeLabel'

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds
########################################################################################################################
for exp_idx, (C, D) in enumerate(product(Cs, Ds), 1):
    log_head: str = f'[{exp_idx}. C={C}; D={D}] '

    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the target file
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_DIR_PATH, f'{C}_encounters_{D}_days_v1.csv')
    df_y: pd.DataFrame = pd.read_csv(y_path)
 
    ####################################################################################################################
    # Perform stratified partitioning
    ####################################################################################################################
    df_y.sort_values(by='PatientDurableKey', inplace=True)
    df_y.reset_index(drop=True, inplace=True)
    y_train, y_test = train_test_split(df_y, test_size=TEST_SIZE, random_state=42, stratify=df_y[target_col])
    print(f'{log_head}[Train] {target_dist(y_train, target_col)}')
    print(f'{log_head}[Test]  {target_dist(y_test, target_col)}')

    ####################################################################################################################
    # Return the target dataset in each partition
    ####################################################################################################################
    y_train_path: str = y_path.replace('_v1.csv', '_train_v2.parquet')
    y_test_path: str = y_path.replace('_v1.csv', '_test_v2.parquet')
    y_train.to_parquet(y_train_path)
    y_test.to_parquet(y_test_path)
    print(f'{log_head}Target datasetes in each partition saved.')
    print('-'*120)